# Exercises

1. Create a materialised view that displays the number of New York City taxi trips per hour. Use the taxi data from the table `nyc_yellow_taxi_trips`. How do you refresh the view if you needed to?
2. Create a formula `rate_per_thousand()` that takes three arguments to calculate the result: `observed_number`, `base_number`, & `decimal_places`.
3. Write a trigger for the `meat_poultry_egg_establishments` table that adds an inspection deadline timestamp six months in the future whenever you insert a new facility into the table. Use the `inspection_deadline` column. You should be able to describe the steps needed to implement a trigger & how the steps relate to each other.

---

# 1.

Let's visualise the `nyc_yellow_taxi_trips` table, just as a refresher.

<img src = "Exercise Images/nyc_yellow_taxi_trips Table.png" width = "600" style = "margin:auto"/>

Let's also write a query that will return the number of New York City taxi trips per hour.

```
SELECT date_part('hour', tpep_pickup_datetime)::integer 
		   AS hour,
	   count(*) AS num_trips
FROM nyc_yellow_taxi_trips
GROUP BY date_part('hour', tpep_pickup_datetime)
ORDER BY date_part('hour', tpep_pickup_datetime);
```

<img src = "Exercise Images/Number of NYC Taxi Trips by Hour.png" width = "600" style = "margin:auto"/>

Great, we'll use this `SELECT` statement in our view. Now we'll create our materialised view.

```
CREATE MATERIALIZED VIEW nyc_taxi_trips_per_hour
AS (
    SELECT date_part('hour', tpep_pickup_datetime)::integer 
    		   AS hour,
    	   count(*) AS num_trips
    FROM nyc_yellow_taxi_trips
    GROUP BY date_part('hour', tpep_pickup_datetime)
    ORDER BY date_part('hour', tpep_pickup_datetime)
);

SELECT * FROM nyc_taxi_trips_per_hour;
```

We can see that it worked, when we go to **Schemas -> public -> Materialized Views** in the object explorer.

<img src = "Exercise Images/Creating a Materialised View.png" width = "600" style = "margin:auto"/>

Say for example, there someone added more trips to the `nyc_yellow_taxi_trips` table. We would need to update our view too. To do that, we would enter the following command:

```
REFRESH MATERIALIZED VIEW nyc_taxi_trips_per_hour;
```

<img src = "Exercise Images/Refreshing a Materialised View.png" width = "600" style = "margin:auto"/>

# 2.

If I'm remembering correctly, to calculate rate per 1000, we use the following formula:

```
rate_per_thousand = (observed_number / base_number) * 1000
```

With this formula in mind, we'll create our function:

```
CREATE OR REPLACE FUNCTION 
rate_per_thousand(observed_number numeric,
				  base_number numeric,
				  decimal_places smallint DEFAULT 1)
RETURNS numeric AS
'SELECT round((observed_number / base_number) * 1000,
			decimal_places);'
LANGUAGE SQL
IMMUTABLE
RETURNS NULL ON NULL INPUT;
```

<img src = "Exercise Images/Creating the rate_per_thousand() Function.png" width = "600" style = "margin:auto"/>

Ok, let's test the function on something. In a previous lesson, we had a query that found the counties that had the highest concentration of businesses per 1,000 population. We'll add the function to it & see if we get a desired result. This was the original query:

```
SELECT cbp.county,
       cbp.st,
       cbp.establishments,
       pop.pop_est_2018,
       round((cbp.establishments::numeric /
           pop.pop_est_2018) * 1000, 1)
           AS estabs_per_1000
FROM cbp_naics_72_establishments AS cbp
JOIN us_counties_pop_est_2019 AS pop
    ON cbp.state_fips = pop.state_fips
        AND cbp.county_fips = pop.county_fips
WHERE pop.pop_est_2018 >= 50000
ORDER BY cbp.establishments::numeric /
    pop.pop_est_2018 DESC;
```

<img src = "Exercise Images/Concentration of Businesses per 1,000 Population.png" width = "600" style = "margin:auto"/>

Now, we'll add our newly created function into the query!

```
SELECT cbp.county,
       cbp.st,
       cbp.establishments,
       pop.pop_est_2018,
       round((cbp.establishments::numeric /
           pop.pop_est_2018) * 1000, 1)
           AS estabs_per_1000,
       rate_per_thousand(cbp.establishments::numeric,
           pop.pop_est_2018) AS rate_per_thou_func
FROM cbp_naics_72_establishments AS cbp
JOIN us_counties_pop_est_2019 AS pop
    ON cbp.state_fips = pop.state_fips
        AND cbp.county_fips = pop.county_fips
WHERE pop.pop_est_2018 >= 50000
ORDER BY cbp.establishments::numeric /
    pop.pop_est_2018 DESC;
```

It worked!

<img src = "Exercise Images/Testing rate_per_thousand() Function.png" width = "600" style = "margin:auto"/>

# 3.

Let's remind ourselves what the `meat_poultry_egg_establishments` table. 

<img src = "Exercise Images/meat_poultry_egg_establishments Table.png" width = "600" style = "margin:auto"/>

First, we'll create the function that will update the inspection deadline upon a new facility insert.

```
CREATE OR REPLACE FUNCTION update_inspect_deadline()
RETURNS trigger AS
$$
BEGIN
    NEW.inspection_deadline := now() +
        '6 months'::interval;
	RETURN NEW;
END;
$$
LANGUAGE plpgsql;
```

Now we'll create the trigger to execute each time a new row is added to the `meat_poultry_egg_establishments_backup` table. We're doing it on the backup table because we don't want to mess up the original table.

```
CREATE TRIGGER estab_insert
BEFORE INSERT ON meat_poultry_egg_establishments_backup
FOR EACH ROW
EXECUTE PROCEDURE update_inspect_deadline();
```

Ok, now we'll insert some new rows into the `meat_poultry_egg_establishments_backup` table. Then, we'll check those new rows.

```
INSERT INTO meat_poultry_egg_establishments_backup (
    establishment_number,
    company    
)
VALUES ('ABC123', 'Beat Your Meat Company'),
       ('XYZ000', 'Choke That Chicken Company'),
       ('TIT303', 'Petting Roosters Company');

SELECT *
FROM meat_poultry_egg_establishments_backup
WHERE company IN ('Beat Your Meat Company',
    'Choke That Chicken Company',
    'Petting Roosters Company');
```

As of running the `SELECT` statement, it is Sunday Dec 22, 2024 11:14 PM PST. So 6 months from now is June 22, 2025 11:14 PM PST.

<img src = "Exercise Images/Creating Trigger for meat_poultry_egg_establishments.png" width = "600" style = "margin:auto"/>

It worked! That's 6 months from now!